In [1]:
import pickle
import numpy as np

toppings_predictor = pickle.load(open('toppings_predictor.pkl', 'rb'))

model = toppings_predictor['model']
scoops_ohe = toppings_predictor['scoops_ohe']
toppings_mlb = toppings_predictor['toppings_mlb']
cone_ohe = toppings_predictor['cone_ohe']
icecream_ohe = toppings_predictor['icecream_ohe']
snack_ohe = toppings_predictor['snack_ohe']


In [2]:
def get_int_between(low, high):
    while True:
        selection = input(": ")
        try:
            selection = int(selection)
        except ValueError:
            continue
        if selection < low or selection > high:
            continue
        return selection


def ask(question, options):
    print(question)
    print(f'Please select one of the following:')
    for i, opt in enumerate(options):
        n = i+1
        print(f'({n}) {opt}')

    selection = get_int_between(1, len(options))-1
    return options[selection]

In [3]:
print("Please enter your age (between 0 and 100)")
age = get_int_between(0, 100)
gender_s = ask("What is your gender?", ["Male", "Female"])
snack_s = ask("How would you best describe your snack preferences?", snack_ohe.categories_[0])
icecream_s = ask("Of the following options, which is your favorite ice cream flavor?", icecream_ohe.categories_[0])
cone_s = ask("Which cup or cone/bowl?", cone_ohe.categories_[0])
scoops = ask("How many scoops?", scoops_ohe.categories_[0])


Please enter your age (between 0 and 100)
What is your gender?
Please select one of the following:
(1) Male
(2) Female
How would you best describe your snack preferences?
Please select one of the following:
(1) salty
(2) savory
(3) sweet
Of the following options, which is your favorite ice cream flavor?
Please select one of the following:
(1) chocolate
(2) cookie dough
(3) cookies and cream
(4) mint
(5) salted caramel or butterscotch
(6) sherbet (any flavor)
(7) strawberry
(8) vanilla
Which cup or cone/bowl?
Please select one of the following:
(1) cake cone/bowl
(2) just a cup, please. No cone/bowl.
(3) sugar cone/bowl
(4) waffle cone/bowl
How many scoops?
Please select one of the following:
(1) 1
(2) 2
(3) 3


In [4]:
# convert to numbers
ismale = np.array([float(gender_s == 'Male')])
age = np.array([age])
snack = snack_ohe.transform([[snack_s]])[0]
icecream = icecream_ohe.transform([[icecream_s]])[0]
cone = cone_ohe.transform([[cone_s]])[0]
scoops = scoops_ohe.transform([[scoops]])[0]

In [5]:
# predict
x_test = np.concatenate([ismale, age, snack, icecream, cone, scoops], axis=0)

y_pred_prob = np.array(model.predict_proba([x_test])).squeeze()[:, 1]
y_pred = y_pred_prob > 0.25
topping_names = toppings_mlb.classes_[y_pred]
topping_proba = y_pred_prob[y_pred]


In [6]:
# display
print("You should try the following toppings (in order):")
for i, (prob, name) in enumerate(sorted(zip(topping_proba, topping_names), reverse=True)):
    print(f"{i+1}. {name} ({round(prob*100)}%)")


You should try the following toppings (in order):
1. M&Ms (or other chocolate candies) (71.0%)
2. sprinkles (66.0%)
3. whipped cream (65.0%)
4. hot fudge (51.0%)
5. gummies (41.0%)
6. cherry on top (31.0%)
